In [1]:
from scipy.stats import kurtosis, skew, entropy
import pandas as pd
import numpy as np

In [2]:
from datasetStuff import expandDataset

In [3]:
data1 = np.load("first_batch_with_labels_likes.npz")
# data2 = np.load("second_batch_with_labels_likes.npz")
# data3 = np.load("third_batch_with_labels_likes.npz")
# data4 = np.load("fourth_batch_with_labels_likes.npz")

In [15]:
eD = expandDataset(data1)

XX1, labels1 = eD.simpleDataRetrieval(data1)
XX2, labels2 = eD.simpleDataRetrieval(data2)
XX3, labels3 = eD.simpleDataRetrieval(data3)
XX4, labels4 = eD.simpleDataRetrieval(data4)

In [4]:
df = expandDataset(data1, True).getDataset()

In [26]:
df.drop([i for i in range(0,999)],axis=1).columns

Index(['average_rating', 'total_interactions', 'likes', 'dislikes', 'meh',
       'neutral_ratings', 'entropy', 'item_mean', 'item_variance', 'skewness',
       'kurtosis', 'likes_ratio', 'dislikes_ratio', 'interaction_balance',
       'neutral_ratio', 'balance_ratio', 'mean', 'std', 'cv',
       'followed majority', 'followed majority %', 'rating_val', 'no_rating',
       'num_pop_liked', 'num_unpop_liked', 'num_pop_disliked',
       'num_unpop_disliked', 'user_x', 'label', 'ld_ratio', 'dm_ratio',
       'lm_ratio', 'pythagoras_ldm', 'pythagoras_ratios', 'log_ldm',
       'pyth_log_ratios', 'follow_rat_val', 'plr_frv', 'cv_foll', 'log_ratios',
       'rating_ratio', 'pyth_log_ldm_ratios', 'user_y', 'modal_ratings',
       'non_modal_ratings', 'rating_diff', 'diff_std', 'diff_mean',
       'mode_ratings_diff', 'items_diff'],
      dtype='object')

In [101]:
allData = pd.DataFrame()

for XX in [XX1,XX2,XX3,XX4]:
    user_features = XX.groupby('user')['item'].agg(
        mean_item_id='mean',
        median_item_id='median',
        item_id_variance='var',
        item_id_std_dev='std',
        item_id_range=lambda x: x.max() - x.min(),
        skewness=skew,
        kurtosis=kurtosis,
        interaction_polarity=lambda x: x.mean() - x.median(),
        mean_difference=lambda x: x.diff().mean(),
        max_difference=lambda x: x.diff().max()
    ).reset_index()
    
    user_features = user_features.sort_values(by='user')
    user_features.index = user_features['user']

    user_features["IQR7525"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
    user_features["IQR8020"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.80) - x.quantile(0.20))

    user_features["rating_entropy"] = XX.groupby('user')['rating'].apply(lambda x: entropy(x.value_counts(normalize=True)))
    allData = pd.concat([allData,user_features])

In [103]:
labels2.index = labels2.user
labels3.index = labels3.user
labels4.index = labels4.user

In [104]:
labels = pd.concat([labels1,labels2,labels3,labels4])
labels.index = labels.user
labels = list(labels['label'])

In [105]:
allData['label'] = labels
allData

,user,mean_item_id,median_item_id,item_id_variance,item_id_std_dev,item_id_range,skewness,kurtosis,interaction_polarity,mean_difference,max_difference,IQR7525,IQR8020,rating_entropy,label
user,,,,,,,,,,,,,,,
0,0,485.341880,513.0,86880.692455,294.755309,958,-0.000684,-1.329244,-27.658120,8.258621,36.0,1.0,1.0,1.017613,1
1,1,66.740741,72.0,1323.430199,36.378980,119,-0.354614,-0.792099,-5.259259,4.576923,21.0,1.0,1.0,0.659153,0
2,2,467.658683,425.0,93221.310512,305.321651,984,0.159742,-1.410857,42.658683,5.927711,31.0,2.0,2.0,1.094542,1
3,3,524.433962,504.5,99399.124169,315.276266,932,-0.015355,-1.566751,19.933962,8.876190,56.0,0.0,0.0,0.514823,0
4,4,440.040000,427.5,76358.826667,276.331009,873,0.033696,-1.391493,12.540000,8.818182,42.0,1.0,1.0,0.686151,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,5095,664.907407,696.5,46627.217680,215.933364,971,-1.274773,1.675145,-31.592593,18.320755,172.0,2.0,2.0,1.091211,0
5096,5096,239.679487,209.5,18755.025808,136.948990,596,0.453369,-0.452917,30.179487,7.740260,56.0,2.0,2.0,1.079378,1
5097,5097,321.333333,311.0,36388.447917,190.757563,823,0.291628,-0.723413,10.333333,2.571875,61.0,1.0,1.0,0.814169,0


In [106]:
allData.to_csv("./csv_files/perfect_feat_all.csv")

In [44]:
data = np.load("fifth_batch_likes.npz")

In [50]:
XX, label = eD.simpleDataRetrieval(data)

In [51]:
XX

,user,item,rating
0,5233,0,1
1,5233,9,0
2,5233,19,1
3,5233,24,0
4,5233,25,0
...,...,...,...
285563,6973,456,0
285564,6973,461,-1
285565,6973,477,1
285566,6973,486,-1


In [54]:
user_features = XX.groupby('user')['item'].agg(
        mean_item_id='mean',
        median_item_id='median',
        item_id_variance='var',
        item_id_std_dev='std',
        item_id_range=lambda x: x.max() - x.min(),
        skewness=skew,
        kurtosis=kurtosis,
        interaction_polarity=lambda x: x.mean() - x.median(),
        mean_difference=lambda x: x.diff().mean(),
        max_difference=lambda x: x.diff().max()
    ).reset_index()
user_features.index = user_features['user']
user_features["IQR7525"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
user_features["IQR8020"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.80) - x.quantile(0.20))

user_features["rating_entropy"] = XX.groupby('user')['rating'].apply(lambda x: entropy(x.value_counts(normalize=True)))

In [55]:
user_features

,user,mean_item_id,median_item_id,item_id_variance,item_id_std_dev,item_id_range,skewness,kurtosis,interaction_polarity,mean_difference,max_difference,IQR7525,IQR8020,rating_entropy
user,,,,,,,,,,,,,,
5100,5100,125.277778,135.0,6971.863492,83.497686,244,-0.051586,-1.527853,-9.722222,6.971429,69.0,1.0,1.0,1.026331
5101,5101,514.985915,544.0,103522.928370,321.749792,984,-0.127482,-1.453571,-29.014085,14.057143,61.0,2.0,2.0,1.090291
5102,5102,472.320442,462.0,98099.663413,313.208658,998,0.067816,-1.417718,10.320442,5.544444,36.0,1.0,1.0,0.721307
5103,5103,379.952381,353.5,54461.499431,233.369877,798,0.125589,-1.246974,26.452381,2.723549,16.0,1.0,1.0,0.873373
5104,5104,194.918033,202.0,15455.009836,124.318180,412,0.080905,-1.159292,-7.081967,6.866667,45.0,0.0,0.0,0.462311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095,7095,442.095238,428.5,73132.953138,270.431051,962,0.168136,-1.116743,13.595238,1.912525,10.0,1.0,1.0,0.966821
7096,7096,383.105263,393.0,58259.276715,241.369585,778,-0.034343,-1.422054,-9.894737,5.893939,40.0,1.0,2.0,1.023867
7097,7097,479.477612,502.0,85488.920746,292.384885,988,-0.029405,-1.267113,-22.522388,4.940000,33.0,1.0,1.0,0.942167


In [56]:
user_features.to_csv("./csv_files/perfect_feat_fifth.csv")